In [1]:
import pandas as pd
data=pd.read_csv('abcnews-date-text.csv',error_bad_lines=False)
#data_text=data[['headline_text']]
#data_text['index']=data_text.index
#documents = data_text
        

In [2]:
data_text = data[['headline_text']]



In [3]:
data_text['index'] = data_text.index
documents = data_text


In [4]:
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [5]:
# Data Preprocessing 
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
               
            

C:\Users\praveen\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\praveen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [7]:
# Select a document to preview after preprocessing
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))



original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfires']


In [8]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0              [decide, community, broadcast, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4             [strike, affect, australian, travellers]
5               [ambitious, olsson, win, triple, jump]
6               [antic, delight, record, break, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                         [australia, lock, timetable]
Name: headline_text, dtype: object

In [9]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 community
2 decide
3 licence
4 aware
5 defamation
6 witness
7 call
8 infrastructure
9 protection
10 summit


In [10]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(112, 1), (489, 1), (970, 1), (4427, 1)]

In [12]:
import pickle
pickle.dump(bow_corpus, open('bow_corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [13]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 112 ("help") appears 1 time.
Word 489 ("rain") appears 1 time.
Word 970 ("bushfires") appears 1 time.
Word 4427 ("dampen") appears 1 time.


In [14]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5955574839932255),
 (1, 0.394834368882524),
 (2, 0.4976240783705052),
 (3, 0.4917188028077664)]


In [15]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)
lda_model.save('model5.gensim')

In [16]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"police" + 0.017*"charge" + 0.016*"court" + 0.012*"attack" + 0.012*"south" + 0.011*"murder" + 0.011*"kill" + 0.011*"death" + 0.010*"face" + 0.010*"woman"
Topic: 1 
Words: 0.010*"say" + 0.009*"country" + 0.009*"health" + 0.008*"fund" + 0.008*"rural" + 0.007*"indigenous" + 0.007*"tasmania" + 0.007*"turnbull" + 0.006*"australia" + 0.006*"fall"
Topic: 2 
Words: 0.017*"trump" + 0.011*"australia" + 0.010*"crash" + 0.009*"world" + 0.008*"live" + 0.008*"donald" + 0.007*"test" + 0.007*"win" + 0.006*"open" + 0.006*"deal"
Topic: 3 
Words: 0.019*"australian" + 0.011*"house" + 0.009*"market" + 0.007*"rise" + 0.007*"perth" + 0.007*"west" + 0.007*"high" + 0.007*"price" + 0.007*"break" + 0.006*"home"
Topic: 4 
Words: 0.015*"government" + 0.013*"queensland" + 0.010*"warn" + 0.009*"say" + 0.009*"plan" + 0.008*"election" + 0.008*"north" + 0.007*"water" + 0.006*"coast" + 0.006*"gold"


In [17]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))


Topic: 0 Word: 0.011*"donald" + 0.009*"weather" + 0.005*"wrap" + 0.005*"islamic" + 0.005*"wild" + 0.005*"know" + 0.005*"andrew" + 0.005*"george" + 0.004*"paul" + 0.004*"wallabies"
Topic: 1 Word: 0.009*"league" + 0.009*"world" + 0.008*"australia" + 0.008*"final" + 0.007*"ash" + 0.006*"rugby" + 0.006*"open" + 0.006*"wednesday" + 0.006*"win" + 0.006*"december"
Topic: 2 Word: 0.012*"market" + 0.009*"queensland" + 0.009*"podcast" + 0.008*"turnbull" + 0.008*"price" + 0.007*"share" + 0.006*"business" + 0.006*"australian" + 0.005*"friday" + 0.005*"dollar"
Topic: 3 Word: 0.015*"police" + 0.014*"crash" + 0.009*"woman" + 0.009*"drum" + 0.008*"die" + 0.008*"death" + 0.008*"miss" + 0.008*"search" + 0.007*"shoot" + 0.007*"charge"
Topic: 4 Word: 0.014*"interview" + 0.009*"christmas" + 0.007*"october" + 0.007*"asylum" + 0.007*"domestic" + 0.006*"violence" + 0.006*"smith" + 0.006*"tony" + 0.005*"wall" + 0.005*"social"
Topic: 5 Word: 0.013*"murder" + 0.013*"charge" + 0.010*"court" + 0.008*"jail" + 0.007

In [18]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfires']

In [19]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3475067913532257	 
Topic: 0.010*"say" + 0.009*"country" + 0.009*"health" + 0.008*"fund" + 0.008*"rural" + 0.007*"indigenous" + 0.007*"tasmania" + 0.007*"turnbull" + 0.006*"australia" + 0.006*"fall"

Score: 0.33151766657829285	 
Topic: 0.015*"government" + 0.013*"queensland" + 0.010*"warn" + 0.009*"say" + 0.009*"plan" + 0.008*"election" + 0.008*"north" + 0.007*"water" + 0.006*"coast" + 0.006*"gold"

Score: 0.23995904624462128	 
Topic: 0.028*"police" + 0.017*"charge" + 0.016*"court" + 0.012*"attack" + 0.012*"south" + 0.011*"murder" + 0.011*"kill" + 0.011*"death" + 0.010*"face" + 0.010*"woman"

Score: 0.040693435817956924	 
Topic: 0.019*"australian" + 0.011*"house" + 0.009*"market" + 0.007*"rise" + 0.007*"perth" + 0.007*"west" + 0.007*"high" + 0.007*"price" + 0.007*"break" + 0.006*"home"

Score: 0.04032309353351593	 
Topic: 0.017*"trump" + 0.011*"australia" + 0.010*"crash" + 0.009*"world" + 0.008*"live" + 0.008*"donald" + 0.007*"test" + 0.007*"win" + 0.006*"open" + 0.006*"deal"


In [20]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.357307493686676	 
Topic: 0.011*"donald" + 0.009*"weather" + 0.005*"wrap" + 0.005*"islamic" + 0.005*"wild" + 0.005*"know" + 0.005*"andrew" + 0.005*"george" + 0.004*"paul" + 0.004*"wallabies"

Score: 0.2925223112106323	 
Topic: 0.015*"police" + 0.014*"crash" + 0.009*"woman" + 0.009*"drum" + 0.008*"die" + 0.008*"death" + 0.008*"miss" + 0.008*"search" + 0.007*"shoot" + 0.007*"charge"

Score: 0.2101520597934723	 
Topic: 0.021*"trump" + 0.014*"rural" + 0.010*"live" + 0.010*"news" + 0.007*"cattle" + 0.007*"marriage" + 0.006*"national" + 0.006*"november" + 0.006*"monday" + 0.005*"station"

Score: 0.02000546082854271	 
Topic: 0.012*"market" + 0.009*"queensland" + 0.009*"podcast" + 0.008*"turnbull" + 0.008*"price" + 0.007*"share" + 0.006*"business" + 0.006*"australian" + 0.005*"friday" + 0.005*"dollar"

Score: 0.020004795864224434	 
Topic: 0.012*"government" + 0.009*"health" + 0.008*"fund" + 0.007*"change" + 0.006*"sport" + 0.006*"plan" + 0.006*"council" + 0.005*"federal" + 0.005*"budg

In [21]:
unseen_document = 'How a Pentagon de to help him identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3666585683822632	 Topic: 0.028*"police" + 0.017*"charge" + 0.016*"court" + 0.012*"attack" + 0.012*"south"
Score: 0.2892063558101654	 Topic: 0.010*"say" + 0.009*"country" + 0.009*"health" + 0.008*"fund" + 0.008*"rural"
Score: 0.27688825130462646	 Topic: 0.015*"government" + 0.013*"queensland" + 0.010*"warn" + 0.009*"say" + 0.009*"plan"
Score: 0.033911313861608505	 Topic: 0.019*"australian" + 0.011*"house" + 0.009*"market" + 0.007*"rise" + 0.007*"perth"
Score: 0.03333551809191704	 Topic: 0.017*"trump" + 0.011*"australia" + 0.010*"crash" + 0.009*"world" + 0.008*"live"


In [22]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('bow_corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')


In [24]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\praveen\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
